# General Eye localization Dataset Processing pipeline

### Standard Library Imports

In [9]:
import sys, os
import hdf5storage
import numpy as np
import shutil
from statistics import mean
from pathlib import Path


### Local Module Imports

In [10]:
sys.path.append(os.path.dirname(os.getcwd()))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

from src.utils.paths_manager import LFPW_RAW_SOURCE_PATH, HELEN_RAW_SOURCE_PATH
from src.dataset.loaders import LFPWDataset, HelenDataset, FaceLandmarkDataloaderContext
from src.dataset.ops import FaceLandmarkImageOps
from src.utils.data_structs import ColumnStruct

### Instantiate Main Context class

In [11]:
dloader_ctx = FaceLandmarkDataloaderContext()

In [12]:
# Dataset source container
dataset_container = [
    LFPWDataset(LFPW_RAW_SOURCE_PATH),
    HelenDataset(HELEN_RAW_SOURCE_PATH)
]

# Tset container 
tset_container = [
    'test',
    'train'
]

In [13]:
# Dataset processing settings
DATASETS_PATH = Path('../datasets')

DESTINATION_PATH = DATASETS_PATH / 'processed' / 'MIRRORED_ITERATION'
TEST_DESTINATION_PATH = DESTINATION_PATH / 'test'
TRAIN_DESTINATION_PATH = DESTINATION_PATH / 'train'

if not DESTINATION_PATH.exists():
    DESTINATION_PATH.mkdir()

if not TEST_DESTINATION_PATH.exists():
    TEST_DESTINATION_PATH.mkdir()

if not TRAIN_DESTINATION_PATH.exists():
    TRAIN_DESTINATION_PATH.mkdir()

In [14]:
shutil.copy('../extras/test/see_bboxes.m', str(TEST_DESTINATION_PATH/'see_bboxes.m'))
shutil.copy('../extras/train/see_bboxes.m', str(TRAIN_DESTINATION_PATH/'see_bboxes.m'))

'..\\datasets\\processed\\MIRRORED_ITERATION\\train\\see_bboxes.m'

In [15]:
STANDARD_FACE_WIDTH = 300
STANDARD_FACE_HEIGHT = 420

In [16]:
combined_container = []

for tset in tset_container:
    for dset in dataset_container:
        dloader_ctx.dloader = dset
        if not dloader_ctx.check_valid():
            print('Invalid Dataset Loader')
            break

        for fl_image in dloader_ctx.load(tset):
            
            orig_fname = os.path.basename(fl_image.fpath)

            # Scale image
            leftmost = min([x[0] for x in fl_image.landmarks.face])
            rightmost = max([x[0] for x in fl_image.landmarks.face])

            face_width = rightmost - leftmost
            scaling_factor = STANDARD_FACE_WIDTH / face_width

            im = fl_image.uniform_scale(scaling_factor)

            # Crop image
            leftmost = min([x[0] for x in im.landmarks.face])
            rightmost = max([x[0] for x in im.landmarks.face])

            face_outline_cluster = im.landmarks.face + im.landmarks.left_brow + im.landmarks.right_brow
            y_median = mean([y[1] for y in face_outline_cluster])

            im = im.crop(left=leftmost,right=rightmost, top=y_median-STANDARD_FACE_HEIGHT/2, bottom=y_median+STANDARD_FACE_HEIGHT/2)

            # Create image with bounding boxes

            bbox_im = im.create_bbox_image()
            bboxed_im = bbox_im.pad_bbox('left_eye', (20, 40, 20, 40)).pad_bbox('right_eye', (20, 40, 20, 40))

            # Create ColumnStruct
            orig_struct = ColumnStruct (filename = orig_fname, header=("filename", "height", "left", "top", "width", "label"), label_by_index=True)
            orig_struct.add(list(bboxed_im.bbox['left_eye']), 1)
            orig_struct.add(list(bboxed_im.bbox['right_eye']), 1)
            combined_container.append(orig_struct.entry())

            bboxed_im.image.save(DESTINATION_PATH / tset / orig_fname)

            print(os.path.basename(fl_image.fpath))

            # Mirrored 
            mirrored_fname = f"{Path(os.path.basename(fl_image.fpath)).stem}_mirrored{Path(os.path.basename(fl_image.fpath)).suffix}"

            mirrored_im = FaceLandmarkImageOps.mirror(im)

             # Create image with bounding boxes

            mirrored_bbox_im = mirrored_im.create_bbox_image()
            mirrored_bboxed_im = mirrored_bbox_im.pad_bbox('left_eye', (20, 40, 20, 40)).pad_bbox('right_eye', (20, 40, 20, 40))

            # Create ColumnStruct
            orig_struct = ColumnStruct (filename = mirrored_fname, header=("filename", "height", "left", "top", "width", "label"), label_by_index=True)
            orig_struct.add(list(mirrored_bboxed_im.bbox['left_eye']), 1)
            orig_struct.add(list(mirrored_bboxed_im.bbox['right_eye']), 1)
            combined_container.append(orig_struct.entry())

            print(os.path.basename(mirrored_fname))

    matrix = {'digitStruct' : np.array([combined_container], dtype=[('name', 'O'), ('bbox', 'O')])}
    hdf5storage.write(matrix, '.', str(DESTINATION_PATH / tset / 'digitStruct.mat') , matlab_compatible=True)

image_0001.png
image_0001_mirrored.png
image_0002.png
image_0002_mirrored.png
image_0003.png
image_0003_mirrored.png
image_0004.png
image_0004_mirrored.png
image_0005.png
image_0005_mirrored.png
image_0006.png
image_0006_mirrored.png
image_0007.png
image_0007_mirrored.png
image_0008.png
image_0008_mirrored.png
image_0009.png
image_0009_mirrored.png
image_0011.png
image_0011_mirrored.png
image_0012.png
image_0012_mirrored.png
image_0013.png
image_0013_mirrored.png
image_0014.png
image_0014_mirrored.png
image_0015.png
image_0015_mirrored.png
image_0016.png
image_0016_mirrored.png
image_0018.png
image_0018_mirrored.png
image_0021.png
image_0021_mirrored.png
image_0022.png
image_0022_mirrored.png
image_0023.png
image_0023_mirrored.png
image_0024.png
image_0024_mirrored.png
image_0025.png
image_0025_mirrored.png
image_0026.png
image_0026_mirrored.png
image_0027.png
image_0027_mirrored.png
image_0028.png
image_0028_mirrored.png
image_0029.png
image_0029_mirrored.png
image_0030.png
image_0030

KeyboardInterrupt: 